In [20]:
import requests
import pandas as pd
import duckdb
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [22]:
polling_2020 = pd.read_csv('https://projects.fivethirtyeight.com/2020-general-data/presidential_polls_2020.csv')

In [93]:
polling_2020['enddate'] = pd.to_datetime(polling_2020['enddate'])
polling_2020.head()

,cycle,state,modeldate,candidate_name,startdate,enddate,pollster,samplesize,population,weight,influence,pct,house_adjusted_pct,trend_and_house_adjusted_pct,tracking,poll_id,question_id
0,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/20/2020,2020-11-01,SurveyMonkey,330.0,lv,0.165557,0.165557,33.0,31.85685,31.85630,T,72663,136416
1,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/18/2020,2020-10-31,SurveyMonkey,361.0,lv,0.025764,0.025605,34.0,32.84583,32.83956,T,72525,136147
2,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/17/2020,2020-10-30,SurveyMonkey,378.0,lv,0.013440,0.013274,33.0,31.85685,31.85175,T,72424,135950
3,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/16/2020,2020-10-29,SurveyMonkey,394.0,lv,0.013962,0.013703,33.0,31.85685,31.85221,T,72345,135785
4,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/8/2020,2020-10-28,University of Wyoming,614.0,lv,0.681785,0.648074,31.0,30.95937,30.97865,NaN,72334,135714


In [94]:
%%sql

SELECT *
FROM polling_2020
LIMIT 5


,cycle,state,modeldate,candidate_name,startdate,enddate,pollster,samplesize,population,weight,influence,pct,house_adjusted_pct,trend_and_house_adjusted_pct,tracking,poll_id,question_id
0,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/20/2020,2020-11-01,SurveyMonkey,330.0,lv,0.165557,0.165557,33.0,31.85685,31.85630,T,72663,136416
1,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/18/2020,2020-10-31,SurveyMonkey,361.0,lv,0.025764,0.025605,34.0,32.84583,32.83956,T,72525,136147
2,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/17/2020,2020-10-30,SurveyMonkey,378.0,lv,0.013440,0.013274,33.0,31.85685,31.85175,T,72424,135950
3,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/16/2020,2020-10-29,SurveyMonkey,394.0,lv,0.013962,0.013703,33.0,31.85685,31.85221,T,72345,135785
4,2020,Wyoming,11/3/2020,Joseph R. Biden Jr.,10/8/2020,2020-10-28,University of Wyoming,614.0,lv,0.681785,0.648074,31.0,30.95937,30.97865,None,72334,135714


In [27]:
%%sql

SELECT distinct candidate_name	
FROM polling_2020
LIMIT 5

,candidate_name
0,Joseph R. Biden Jr.
1,Donald Trump


In [95]:
%%sql
SELECT *
FROM
    (SELECT 
        state,
        CAST(enddate as DATE) as enddate,
        poll_id, 
        question_id,
        row_number() OVER (PARTITION BY state ORDER BY CAST(enddate as DATE) DESC) as poll_number,
        SUM(CASE WHEN candidate_name = 'Joseph R. Biden Jr.' then pct else 0 END) as BIDEN,
        SUM(CASE WHEN candidate_name = 'Donald Trump' then pct else 0 END) as TRUMP,
    FROM polling_2020
    GROUP BY 
        state,
        enddate,
        poll_id, 
        question_id
    )
WHERE state = 'Wyoming'
ORDER BY CAST(enddate as DATE) DESC

,state,enddate,poll_id,question_id,poll_number,BIDEN,TRUMP
0,Wyoming,2020-11-01,72663,136416,1,33.00,66.00
1,Wyoming,2020-10-31,72525,136147,2,34.00,65.00
2,Wyoming,2020-10-30,72424,135950,3,33.00,65.00
3,Wyoming,2020-10-29,72345,135785,4,33.00,66.00
4,Wyoming,2020-10-28,72334,135714,5,31.00,59.00
5,Wyoming,2020-10-28,72242,135582,6,31.00,68.00
6,Wyoming,2020-10-27,72088,135215,7,32.00,66.00
7,Wyoming,2020-10-25,71978,134981,8,31.00,67.00
8,Wyoming,2020-10-24,71893,134807,9,31.00,67.00
9,Wyoming,2020-10-23,71826,134672,10,31.00,67.00


In [101]:
query = """
SELECT *
FROM
    (SELECT 
        state,
        enddate,
        poll_id, 
        question_id,
        row_number() OVER (PARTITION BY state ORDER BY enddate DESC) as poll_number,
        SUM(CASE WHEN candidate_name = 'Joseph R. Biden Jr.' then pct else 0 END) as BIDEN,
        SUM(CASE WHEN candidate_name = 'Donald Trump' then pct else 0 END) as TRUMP,
    FROM polling_2020
    GROUP BY 
        state,
        enddate,
        poll_id, 
        question_id
    )
WHERE poll_number <= 15
"""

In [102]:
result = conn.execute(query)

In [103]:
polling_2020_updated = pd.DataFrame(result.fetchall(), columns=[x[0] for x in result.description])

In [104]:
polling_2020_updated.head()

,state,enddate,poll_id,question_id,poll_number,BIDEN,TRUMP
0,New Mexico,2020-11-01,72681,136434,1,56.0,42.0
1,New Mexico,2020-10-31,72543,136165,2,55.0,43.0
2,New Mexico,2020-10-30,72442,135968,3,55.0,43.0
3,New Mexico,2020-10-29,72400,136015,4,54.0,42.0
4,New Mexico,2020-10-29,72363,135803,5,56.0,43.0


In [107]:
polling_2020_updated.loc[polling_2020_updated.state == 'Pennsylvania']

,state,enddate,poll_id,question_id,poll_number,BIDEN,TRUMP
797,Pennsylvania,2020-11-02,72647,136322,1,48.4,49.2
798,Pennsylvania,2020-11-02,72773,136555,2,49.9,48.4
799,Pennsylvania,2020-11-01,72638,136309,3,51.0,49.0
800,Pennsylvania,2020-11-01,72629,136293,4,50.0,44.0
801,Pennsylvania,2020-11-01,72640,136312,5,50.0,47.0
802,Pennsylvania,2020-11-01,72507,136069,6,51.5,45.5
803,Pennsylvania,2020-11-01,72719,136488,7,52.0,45.0
804,Pennsylvania,2020-11-01,72650,136326,8,51.0,46.0
805,Pennsylvania,2020-11-01,72730,136501,9,50.0,46.0
806,Pennsylvania,2020-11-01,72675,136428,10,51.0,47.0


In [108]:
polling_2024_url = 'https://projects.fivethirtyeight.com/polls-page/data/president_polls.csv'

In [110]:
polling_2024_initial = pd.read_csv(polling_2024_url)

In [111]:
%%sql

SELECT *
FROM polling_2024_initial
LIMIT 5

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,86653,1424,Siena/NYT,None,None,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,45.0
1,86653,1424,Siena/NYT,None,None,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,46.0
2,86653,1424,Siena/NYT,None,None,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,40.0
3,86653,1424,Siena/NYT,None,None,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,42.0
4,86653,1424,Siena/NYT,None,None,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,IND,Kennedy,31042,Robert F. Kennedy,2.0


In [113]:
polling_2024_initial.dtypes
polling_2024_initial['end_date'] = pd.to_datetime(polling_2024_initial['end_date'])

C:\Users\appar\AppData\Local\Temp\ipykernel_25716\349816578.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  polling_2024_initial['end_date'] = pd.to_datetime(polling_2024_initial['end_date'])


In [115]:
polling_2024_initial.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,86653,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,45.0
1,86653,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,46.0
2,86653,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,40.0
3,86653,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,42.0
4,86653,1424,Siena/NYT,NaN,NaN,The New York Times/Siena College,448,The New York Times/Siena College,3.0,-1.5,...,11/5/24,general,False,False,NaN,IND,Kennedy,31042,Robert F. Kennedy,2.0


In [120]:
%%sql
SELECT *
FROM
    (SELECT 
        state,
        CAST(end_date as DATE) as enddate,
        poll_id, 
        question_id,
        row_number() OVER (PARTITION BY state ORDER BY CAST(end_date as DATE) DESC) as poll_number,
        SUM(CASE WHEN candidate_id = 19368 then pct else 0 END) as BIDEN,
        SUM(CASE WHEN candidate_id = 16651 then pct else 0 END) as TRUMP,
    FROM polling_2024_initial
    GROUP BY 
        state,
        end_date,
        poll_id, 
        question_id
    )
WHERE state = 'Michigan'
    AND poll_number <= 15
ORDER BY CAST(enddate as DATE) DESC

,state,enddate,poll_id,question_id,poll_number,BIDEN,TRUMP
0,Michigan,2024-03-28,86603,196031,1,39.8,43.0
1,Michigan,2024-03-28,86603,196030,2,43.7,48.0
2,Michigan,2024-03-24,86584,195855,3,45.0,48.0
3,Michigan,2024-03-24,86584,195841,4,37.0,39.0
4,Michigan,2024-03-24,86584,195848,5,40.0,45.0
5,Michigan,2024-03-19,86662,196535,6,45.0,51.0
6,Michigan,2024-03-18,86509,194890,7,42.0,50.0
7,Michigan,2024-03-18,86502,194847,8,40.6,43.2
8,Michigan,2024-03-18,86502,194846,9,49.7,50.3
9,Michigan,2024-03-18,86509,194891,10,34.0,40.0


In [121]:
query =  """
SELECT *
FROM
    (SELECT 
        state,
        CAST(end_date as DATE) as enddate,
        poll_id, 
        question_id,
        row_number() OVER (PARTITION BY state ORDER BY CAST(end_date as DATE) DESC) as poll_number,
        SUM(CASE WHEN candidate_id = 19368 then pct else 0 END) as BIDEN,
        SUM(CASE WHEN candidate_id = 16651 then pct else 0 END) as TRUMP,
    FROM polling_2024_initial
    GROUP BY 
        state,
        end_date,
        poll_id, 
        question_id
    )
WHERE state = 'Michigan'
    AND poll_number <= 15
ORDER BY CAST(enddate as DATE) DESC
"""

In [122]:
result = conn.execute(query)

In [123]:
polling_2024_updated = pd.DataFrame(result.fetchall(), columns=[x[0] for x in result.description])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [125]:
polling_2024_updated.head()

,state,enddate,poll_id,question_id,poll_number,BIDEN,TRUMP
0,Michigan,2024-03-28,86603,196030,1,43.7,48.0
1,Michigan,2024-03-28,86603,196031,2,39.8,43.0
2,Michigan,2024-03-24,86584,195841,3,37.0,39.0
3,Michigan,2024-03-24,86584,195848,4,40.0,45.0
4,Michigan,2024-03-24,86584,195855,5,45.0,48.0
